In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

In [2]:
final_Df = pd.read_csv('C:/Users/acer/PycharmProjects/Extenso_Assignment/Entity_Match/project/files/output_files/final_output.csv')

In [5]:
print(final_Df)

    votersID  votersAge  SC Number                      Permanent_Address  \
0   11116874         45  001.01.01            Baluwatar, Kathmandu, Nepal   
1   22259363         38  001.01.02               Lakeside, Pokhara, Nepal   
2   33385241         52  001.01.03  Chitwan National Park, Chitwan, Nepal   
3   44475489         30  001.01.04              Biratnagar, Morang, Nepal   
4   55562139         27  001.01.05              Bharatpur, Chitwan, Nepal   
5   66698214         41  001.01.06               Butwal, Rupandehi, Nepal   
6   77714635         36  001.01.07              Hetauda, Makwanpur, Nepal   
7   88847326         50  001.01.08               Janakpur, Dhanusa, Nepal   
8   99953421         43  001.01.09                Nepalgunj, Banke, Nepal   
9   10103847         34  001.01.10                 Dharan, Sunsari, Nepal   
10  11113982         39  001.01.11                Birtamode, Jhapa, Nepal   
11  12122547         48  001.01.12              Dhangadhi, Kailali, Nepal   

In [ ]:
final_Df

In [16]:
def get_file_names(directory):
    # Get the list of files in the specified directory
    files = os.listdir(directory)
    
    # Extract only the file names without the extensions
    file_names = [os.path.splitext(file)[0] for file in files]
    
    return file_names

In [17]:
name = get_file_names('C:/Users/acer/PycharmProjects/Extenso_Assignment/Entity_Match/project/files/input_files')

In [18]:
name

In [19]:
def load_df(filename, file_path='../data'):
    full_path = os.path.join(file_path, f'{filename}.csv')
    try:
        df = pd.read_csv(full_path)
        print(f"File '{filename}.csv' loaded successfully from '{file_path}'.")
        return df
    except FileNotFoundError:
        print(f"Error: The file '{filename}.csv' was not found in the directory '{file_path}'.")
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{filename}.csv' is empty.")
    except pd.errors.ParserError:
        print(f"Error: The file '{filename}.csv' could not be parsed.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [20]:
layouts = []
for i,j in enumerate(name):

In [25]:
def load_df_from_dir(dir_path):
    csv_files = [f for f in os.listdir(dir_path) if f.endswith(".csv")]
    layouts = []

    for csv_file in csv_files:
        file_path = os.path.join(dir_path, csv_file)
        df = pd.read_csv(file_path)
        layouts.append(df)
    
    return layouts

In [26]:
layouts = load_df_from_dir('C:/Users/acer/PycharmProjects/Extenso_Assignment/Entity_Match/project/files/input_files')

In [28]:
layouts[0]

In [29]:
def sanitize(df):
    return df.map(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else '' if pd.isna(x) else x)

In [30]:
layout_copies = []
for layout in layouts:
    try:
        if layout is not None:
            sanitized_layout = sanitize(layout.copy())
            layout_copies.append(sanitized_layout)
        else:
            layout_copies.append(None)
    except Exception as e:
        print(f"An error occurred while sanitizing a layout: {e}")
        layout_copies.append(None)

In [31]:
for i in range(len(layout_copies)):
    layout_copies[i] = sanitize(layout_copies[i])

In [32]:
def create_soup(df, df_, soup, soup_name):
    df[soup_name] = df_[soup].apply(lambda x: ' '.join(x.values.astype(str)).lower(), axis=1)

soup = ['Name', 'Date of Birth', 'Father_Name']

for i, j, k, in zip(layouts, layout_copies, range(len(layouts))):
    create_soup(i, j, soup, f"soup{k+1}")

In [33]:
def combine(A, B, soup_A, soup_B, threshold=0.3):
    # Initialize the TF-IDF Vectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Combine the textual data for fitting the TF-IDF model
    combined_soup = pd.concat([A[soup_A], B[soup_B]], ignore_index=True)
    tfidf.fit(combined_soup)
    
    # Transform the textual data into TF-IDF matrices
    tfidf_matrix_A = tfidf.transform(A[soup_A])
    tfidf_matrix_B = tfidf.transform(B[soup_B])
    
    # Compute cosine similarity between the two matrices
    similarity = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)
    similarity_df = pd.DataFrame(similarity, index=A.index, columns=B.index)
    
    # Determine the index of the most similar row in B for each row in A
    max_idx_row = similarity_df.idxmax(axis=1)
    similarity_mask = similarity_df.max(axis=1) >= threshold
    
    # Initialize the combined DataFrame with A, ensuring all columns from both DataFrames
    combined_columns = list(set(A.columns) | set(B.columns))
    combined_data = pd.DataFrame(columns=combined_columns)
    
    # Merge the similar rows 
    for idx_A in A.index:
        if similarity_mask[idx_A]:
            idx_B = max_idx_row[idx_A]
            combined_row = A.loc[idx_A].combine_first(B.loc[idx_B])
        else:
            combined_row = A.loc[idx_A]
        combined_data = pd.concat([combined_data, combined_row.to_frame().T], ignore_index=True)
    
    # Append non-similar rows from B to A
    new_records = B.loc[~B.index.isin(max_idx_row[similarity_mask].values)]
    result = pd.concat([combined_data, new_records], ignore_index=True)
    result.drop(columns=soup_B, inplace=True)
    return result

In [36]:
result_12 = combine(layouts[0], layouts[1], 'soup1', 'soup2')
result_123 = combine(result_12, layouts[2], 'soup1', 'soup3')
result_1234 = combine(result_123, layouts[3], 'soup1', 'soup4')
final_result = combine(result_1234, layouts[4], 'soup1', 'soup5')

In [37]:
final_result